In [532]:
import json, random
import numpy as np
from pprint import pprint
from collections import Counter, defaultdict
from bs4 import BeautifulSoup
import requests
import urllib.request
import re
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import pickle
import copy
import spacy
from spacy import displacy
np.set_printoptions(precision=4)

In [2]:
txt_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_J.json", "r"))
print(len(txt_dataset))

7921


In [506]:
# Randomly look at positive facts and the wikipedia pages they come from
analyze_ids = random.sample(range(7921), 15)
print(analyze_ids)
for i in analyze_ids:
    k = str(i)
    print('Q: ', txt_dataset[k]['Q'])
    print('A: ', txt_dataset[k]['A'])
    for f in txt_dataset[k]['SupportingFacts']:
        print(f['fact'])
        print(f['url'])
    print()

[4330, 982, 5834, 6850, 3781, 697, 3929, 2376, 5909, 4968, 5537, 1581, 6812, 6693, 2102]
Q:  Did Nickelodeon or Netflix premiere Winx episodes first?
A:  Nickelodeon was first as it premiered the Winx Club series in 2012, whereas Netflix premiered the Winx Saga in 2021.
Its television premiere was on 11 March 2012 on Nickelodeon in the United States. The plot takes place after the events of the first three seasons. In 2007, production began on a sequel to The Secret of the Lost Kingdom, before the fourth season had been written. It was released in Italy on 29 October 2010.
https://en.wikipedia.org/wiki/List_of_Winx_Club_episodes
The six-episode first season debuted on Netflix on 22 January 2021 to mixed reviews. In February 2021, the series was renewed for a second season. Bloom, a fairy with fire powers, enrolls at a magical boarding school in the Otherworld called Alfea College.
https://en.wikipedia.org/wiki/Fate:_The_Winx_Saga

Q:  What is a concept that is centrally emphasized by b

Step 1: Question --> Identify salient spans

Step 2: Spans --> Wiki pages

Step 3: Wikipages --> html parsing --> paragraphs, images & their captions

Step 4: Choose paragraphs with high overlap with the question **without** giving away the core answer info

#### Step 1: Question --> Identify salient spans

In [526]:
nlp = spacy.load('en_core_web_sm')

In [527]:
def show_ents(doc): 
    if doc.ents: 
        for ent in doc.ents: 
            print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
    else: print('No named entities found.')

In [543]:
wikipedia.search("1854 senate")

['1854 and 1855 United States Senate elections',
 'Kansas–Nebraska Act',
 '1974 United States Senate election in Vermont',
 'List of elections in the United States',
 'Chamber of Deputies of the Dominican Republic',
 '1854 United States elections',
 '1852 and 1853 United States Senate elections',
 'Argentine Senate',
 'Bleeding Kansas',
 '2022 United States Senate election in Vermont']

In [541]:
k = str(random.choice(range(7921)))
print(k)
Q = txt_dataset[k]['Q']
print('Q: ', Q)
print('A: ', txt_dataset[k]['A'])
for f in txt_dataset[k]['SupportingFacts']:
    print(f['fact'])
    print(f['url'])
print("\nEntities:")
doc = nlp(Q)
show_ents(doc)
displacy.render(doc, style='ent', jupyter=True)
for e in doc.ents:
    print(wikipedia.search(e.text))

1994
Q:  Which party controlled the Senate and the Presidency after the 1854 elections?
A:  The Democrats controlled the Senate and the Democrat Franklin Pierce was President once the 1854 election was concluded.
Nathaniel Banks, a member of the American Party and the Free Soil Party, won election as Speaker of the House after a protracted battle, defeating Democrat William Aiken. In the Senate, Democrats retained a strong majority, while the Opposition replaced the Whigs as the second largest party in the chamber.
https://en.wikipedia.org/wiki/United_States_elections,_1854
The 1854 United States elections was the midterm election choosing members of the 32nd United States Congress during the middle of Democratic President Franklin Pierce 's term.
https://en.wikipedia.org/wiki/1854_United_States_elections

Entities:
Senate - 27 - 33 - ORG - Companies, agencies, institutions, etc.
1854 - 63 - 67 - DATE - Absolute or relative dates or periods


['Senate', 'United States Senate', 'List of current United States senators', '2022 United States Senate elections', 'Senate of Canada', 'Filibuster in the United States Senate', 'Roman Senate', 'Senate of the Philippines', 'Illinois Senate', 'Senate of Uruguay']
['1854', 'Siege of Sevastopol (1854–1855)', 'Miao Rebellion (1854–1873)', 'Crimean War', '1854 in Australia', 'NGC 1854', 'Oattathoodhuvan-1854', '1854 in France', '1854 in sports', 'Kansas–Nebraska Act']


In [19]:
USER_AGENT_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
            "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
            "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
        ]

In [85]:
url_blocklist = ['seal', 'sign ', 'pdf', 'gif', 'icon', 'notice', 'cartoon', 'publish', 'menu', 'logo', 'svg', 'webm', 'page', \
                     'ogg', 'flickr', 'poster', 'ogv', 'banner', 'tif', 'montage', 'centralautologin', 'footer']

In [495]:
### Scrap imgs and their captions. 
url = 'https://en.wikipedia.org/wiki/Gulf_of_Mexico'
req = urllib.request.Request(url, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
with urllib.request.urlopen(req) as f:
    html = f.read().decode('utf-8')

soup = BeautifulSoup(html, 'html.parser')
#links = soup.find_all('img', {'src':re.compile('.jpg|.png')})
links = soup.find_all('img')
print(len(links))
for l in links:
    imgUrl = l.get('src')
    #if "http" not in imgUrl: imgUrl = "https://" + url.split("/")[2] + imgUrl
    if int(l['width'])<100 or int(l['height'])<100:
        continue
    if any(b in imgUrl.lower() for b in url_blocklist): 
        continue
    print(" --------------------------------------------------------------- ")
    print(imgUrl)
    
    # Special case for thumb images, which are located inside a table
    thumbinner_div = l.find_parent("div", class_='thumbinner')
    if thumbinner_div: 
        print("parent thumbinner<div>: ", thumbinner_div.text)
        continue
        
    tr_parent = l.find_parent('tr')
    if tr_parent: print("parent<tr>: ", tr_parent.text)
        
    next_p = l.find_next('p')
    if next_p: print("<p>: ", next_p.text)
                
    prev_th = l.find_previous('th')
    if prev_th: print("prev<th>: ", prev_th.text)
    
    prev_p = l.find_previous('p')
    if prev_p: print("prev<p>: ", prev_p.text)


46
 --------------------------------------------------------------- 
//upload.wikimedia.org/wikipedia/commons/thumb/4/47/Gfp-texas-galveston-island-state-park-inlet-shore.jpg/264px-Gfp-texas-galveston-island-state-park-inlet-shore.jpg
parent<tr>:  Gulf of Mexico coastline near Galveston, Texas
<p>:  The Gulf of Mexico (Spanish: Golfo de México) is an ocean basin and a marginal sea of the Atlantic Ocean,[1] largely surrounded by the North American continent.[2] It is bounded on the northeast, north and northwest by the Gulf Coast of the United States; on the southwest and south by the Mexican states of Tamaulipas, Veracruz, Tabasco, Campeche, Yucatan, and Quintana Roo; and on the southeast by Cuba. The US states of Texas, Louisiana, Mississippi, Alabama, and Florida, which border the Gulf on the north, are often referred to as the "Third Coast" of the United States (in addition to its Atlantic and Pacific coasts). 

prev<th>:  Gulf of Mexico
prev<p>:  

 --------------------------------

In [489]:
### Given important spans, find relevant Wikipedia pages
import wikipedia
ny = wikipedia.search("Ferrocerium")
#print(ny.title)
#print(ny.url)
#print(ny.content)

In [490]:
ny

['Ferrocerium',
 'Flint',
 'Fire making',
 'List of named alloys',
 'Mischmetal',
 'Cerium',
 'Glossary of firelighting',
 'Lighter',
 'Neodymium magnet',
 'Tinder']